In [3]:
import os
import sys
sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"

import quantfreedom as qf
import pandas as pd
import numpy as np
import numba
from quantfreedom.backtester.base.base import rsi_below_entries, log_trades_plot, rsi_above_entries
from quantfreedom.backtester.nb.simulate import simulate_from_signals
from quantfreedom.backtester.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
)
pd.options.display.float_format = '{:,.2f}'.format
# qf.clear_cache()

prices = pd.read_csv('30min.csv', index_col='time')
entries = rsi_below_entries(
    timeperiods=[20],
    below_ranges=[40],
    prices=prices.close
)


In [4]:
df, order_rec, cart = simulate_from_signals(
    open_prices=prices.open.values,
    high_prices=prices.high.values,
    low_prices=prices.low.values,
    close_prices=prices.close.values,
    entries=entries.values,
    equity=1000.,
    fee_pct=.06,
    mmr=.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=3.,
    risk_rewards=[3,5],
    size_pct=1.,
    sl_pcts=[4, 5],
    # tsl_true_or_false=True,
    # tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    # tsl_trail_by_pct=1,
    # tsl_when_pct_from_avg_entry=[3,4],
)


In [5]:
df_or = pd.DataFrame(order_rec)
for i in range(len(OrderType._fields)):
    df_or.replace({'order_type': {i: OrderType._fields[i]}}, inplace=True)
df_or.head(20)
    

,order_id,or_set,ind_set,bar,size_value,price,avg_entry,fees_paid,order_type,real_pnl,equity,sl_prices,tsl_prices,tp_prices
0,0,0,0,31,242.86,"10,850.00","10,850.00",NaN,LongEntry,NaN,"1,000.00","10,416.00",NaN,"12,204.11"
1,1,0,0,32,242.86,"10,725.00","10,787.14",NaN,LongEntry,NaN,"1,000.00","10,355.65",NaN,"12,133.40"
2,2,0,0,33,242.86,"10,577.50","10,716.34",NaN,LongEntry,NaN,"1,000.00","10,287.69",NaN,"12,053.77"
3,3,0,0,947,728.58,"12,053.77","10,716.34",0.93,LongTP,90.00,"1,090.00","10,287.69",NaN,"12,053.77"
4,4,0,0,1089,264.72,"12,846.00","12,846.00",NaN,LongEntry,NaN,"1,090.00","12,332.16",NaN,"14,449.22"
5,5,0,0,1090,264.72,"12,803.00","12,824.46",NaN,LongEntry,NaN,"1,090.00","12,311.49",NaN,"14,424.99"
6,6,0,0,1091,264.72,"12,812.00","12,820.31",NaN,LongEntry,NaN,"1,090.00","12,307.49",NaN,"14,420.32"
7,7,0,0,1694,794.15,"14,420.32","12,820.31",1.01,LongTP,98.10,"1,188.10","12,307.49",NaN,"14,420.32"
8,8,0,0,1805,288.54,"15,441.00","15,441.00",NaN,LongEntry,NaN,"1,188.10","14,823.36",NaN,"17,368.08"
9,9,0,0,1808,288.54,"15,344.50","15,392.60",NaN,LongEntry,NaN,"1,188.10","14,776.89",NaN,"17,313.64"


In [6]:
pd.DataFrame(df)


,or_set,ind_set,win_rate,gains_pct,to_the_upside,total_trades,total_BE,total_fees,total_pnl,ending_eq
0,0,0,34.78,71.80,0.49,46.00,0,49.36,717.99,"1,717.99"
1,1,0,39.39,70.15,0.23,33.00,0,32.35,701.54,"1,701.54"
2,2,0,31.58,171.15,0.63,38.00,0,55.95,"1,711.51","2,711.51"
3,3,0,34.78,97.71,0.42,23.00,0,25.05,977.07,"1,977.07"


In [7]:
df_cart = pd.DataFrame(cart).dropna(axis='columns', thresh=1)
for i in range(len(SL_BE_or_Trail_BasedOn._fields)):
    df_cart.replace({'tsl_based_on': {i: SL_BE_or_Trail_BasedOn._fields[i]}}, inplace=True)
    df_cart.replace({'sl_to_be_based_on': {i: SL_BE_or_Trail_BasedOn._fields[i]}}, inplace=True)
df_cart.T

,0,1,2,3
order_settings_id,0.00,1.00,2.00,3.00
max_equity_risk_pct,3.00,3.00,3.00,3.00
risk_rewards,3.00,3.00,5.00,5.00
size_pct,1.00,1.00,1.00,1.00
sl_pcts,4.00,5.00,4.00,5.00


In [8]:
log_trades_plot(
    open_prices=prices.open,
    high_prices=prices.high,
    low_prices=prices.low,
    close_prices=prices.close,
    order_records=order_rec[0:30],
)
